In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [52]:
df = pd.read_csv('tweets_stocks_combined_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2669 entries, 0 to 2668
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2669 non-null   float64
 1   cleaned_text           2660 non-null   object 
 2   favorites              2669 non-null   int64  
 3   retweets               2669 non-null   int64  
 4   date                   2669 non-null   object 
 5   tweet_datetime         2669 non-null   object 
 6   date_part              2669 non-null   object 
 7   time_part              2669 non-null   object 
 8   hour                   2669 non-null   int64  
 9   year                   2669 non-null   int64  
 10  month                  2669 non-null   int64  
 11  sentiment_score        2669 non-null   object 
 12  tweet_length           2669 non-null   int64  
 13  compound               2669 non-null   float64
 14  entity                 2669 non-null   object 
 15  enti

In [53]:
model_a_df = df.dropna(subset=['cleaned_text'])

In [54]:
model_a_df

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,...,compound,entity,entity_type,emoji_free_tweets,url_free_tweets,datetime_5mins_after,price_5mins_after,5mins_price_diff_abs,5mins_price_diff_perc,prev_30mins_prices
0,9.353400e+17,thank you rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,...,0.4199,['Rand'],['ORG'],thank you rand!,thank you rand!,2017-11-28 10:55:00,261.085000,-0.015000,-0.000057,"[260.96, 261.03, 261.01, 261.015, 261.04, 261...."
1,8.997980e+17,join me live from fort myer in arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,...,0.2960,"['Fort Myer', 'Arlington', 'Virginia', '➡']","['GPE', 'GPE', 'GPE', 'ORG']","join me live from fort myer in arlington, virg...","join me live from fort myer in arlington, virg...",2017-08-22 09:05:00,243.670000,0.000000,0.000000,"[243.76, 243.79, 243.85, 243.86, 243.81, 243.7..."
2,8.939700e+17,thank you nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,...,0.4199,['Nicole'],['PERSON'],thank you nicole!,thank you nicole!,2017-05-09 07:06:00,239.920000,0.045000,0.000188,"[239.73, 239.73, 239.73, 239.73, 239.73, 239.7..."
3,8.819770e+17,thank you to shawn steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,...,0.6486,['Shawn Steel'],['PERSON'],thank you to shawn steel for the nice words on...,thank you to shawn steel for the nice words on...,2017-03-08 04:49:00,236.913333,0.033333,0.000141,"[236.84, 236.84, 236.84, 236.8533333333333, 23..."
4,8.778460e+17,great night in iowa special people thank you,56446,8039,2017-06-22 11:11:00,2017-06-22 19:11:00,2017-06-22,19:11:00,19,2017,...,0.8622,"['night', 'Iowa']","['TIME', 'GPE']",great night in iowa - special people. thank you!,great night in iowa - special people. thank you!,2017-06-22 19:16:00,242.905000,0.025000,0.000103,"[242.85, 242.85, 242.85, 242.85, 242.85, 242.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,9.990960e+17,if the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,...,0.6792,"['Collusion', 'Russia', 'Crooked Hillary', 'Be...","['ORG', 'GPE', 'PERSON', 'PERSON']",if the person placed very early into my campai...,if the person placed very early into my campai...,2018-05-23 09:18:00,271.100000,0.060000,0.000221,"[271.18, 271.16, 271.18, 271.15, 271.08, 271.0..."
2665,9.874600e+17,so general michael flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,...,-0.6689,"['Michael Flynn', 'Shadey James Comey', 'Leak'...","['PERSON', 'PERSON', 'PERSON', 'ORDINAL', 'GPE...",so general michael flynn’s life can be totally...,so general michael flynn’s life can be totally...,2018-04-20 18:39:00,266.866667,0.046667,0.000175,"[266.74625000000003, 266.7475, 266.74875, 266...."
2666,9.870960e+17,my thoughts prayers and condolences are with t...,62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,...,0.6900,"['two', '@GCSOFlorida', 'HEROES', 'today', 'Sa...","['CARDINAL', 'ORG', 'ORG', 'DATE', 'PERSON', '...","my thoughts, prayers and condolences are with ...","my thoughts, prayers and condolences are with ...",2018-04-20 06:35:00,268.715000,0.095000,0.000354,"[268.77, 268.74333333333334, 268.7166666666666..."
2667,9.863570e+17,today’s court decision means that congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,...,-0.9493,"['Today', 'Congress', 'Congress', 'House', 'Se...","['DATE', 'ORG', 'ORG', 'ORG', 'ORG', 'ORG']",today’s court decision means that congress mus...,today’s court decision means that congress mus...,2018-04-18 05:39:00,270.600000,0.000000,0.000000,"[270.73, 270.7, 270.7275, 270.755, 270.7825, 2..."


# Model A (only word vectors)

In [58]:
model_a_X = model_a_df.loc[:, 'cleaned_text']
model_a_y = model_a_df.loc[:, '5mins_price_diff_perc']

In [59]:
model_a_X_train, model_a_X_test, model_a_y_train, model_a_y_test = train_test_split(model_a_X, model_a_y, test_size=0.33, random_state=42)

In [60]:
model_a_corpus_list = []

for i in model_a_X_train:
    model_a_corpus_list.append(i.split())

In [61]:
model_a_word2vec_model = Word2Vec(model_a_corpus_list, min_count=1, size=100)
model_a_pretrained_weights = word2vec_model.wv.vectors

In [62]:
model_a_num_words = [len(i) for i in model_a_corpus_list]
model_a_longest_sentence_len = max(model_a_num_words)

In [63]:
def sentence_to_indices_padded(sentences, longest_sentence_len):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [64]:
model_a_X_train_padded = sentence_to_indices_padded(model_a_X_train, model_a_longest_sentence_len)
model_a_X_test_padded = sentence_to_indices_padded(model_a_X_test, model_a_longest_sentence_len)

In [ ]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
w2c = dict()
for item in word2vec_model.wv.vocab:
    w2c[item]=word2vec_model.wv.vocab[item].count
w2c_sorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
print(w2c_sorted)

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
words_w2v = list(w2c.keys())
words_gloves = list(glove_word_count.keys())
extra = []\
for word in words_w2v:
    if word not in words_gloves:
        extra.append(word)
print(extra)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
vocab_size_glove, embedding_size_glove = embedding_matrix.shape
print(vocab_size_glove)
print(embedding_size)
print(count)
print(skipped_words)
embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=longest_sentence_len,
                            trainable=False)

In [65]:
def create_model_a(pretrained_weights, longest_sentence_len):
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [66]:
model_a = create_model_a(model_a_pretrained_weights, model_a_longest_sentence_len)

In [67]:
model_a.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [68]:
model_a.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 157, 100)          589300    
_________________________________________________________________
LSTM1 (LSTM)                 (None, 157, 4)            1680      
_________________________________________________________________
Dropout1 (Dropout)           (None, 157, 4)            0         
_________________________________________________________________
LSTM2 (LSTM)                 (None, 4)                 144       
_________________________________________________________________
Dropout2 (Dropout)           (None, 4)                 0         
_________________________________________________________________
Dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dropout_6 (Dropout)          (None, 4)               

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

Train on 1425 samples, validate on 357 samples
Epoch 1/50
1408/1425 [============================>.] - ETA: 0s - loss: 0.0349 - mean_absolute_error: 0.1091
Epoch 00001: val_loss improved from inf to 0.00001, saving model to ./model_a_checkpoint/09112020 1115h.h5
1425/1425 [==============================] - 24s 17ms/sample - loss: 0.0347 - mean_absolute_error: 0.1088 - val_loss: 1.4224e-05 - val_mean_absolute_error: 0.0038
Epoch 2/50
1408/1425 [============================>.] - ETA: 0s - loss: 0.0253 - mean_absolute_error: 0.0930
Epoch 00002: val_loss did not improve from 0.00001
1425/1425 [==============================] - 18s 12ms/sample - loss: 0.0252 - mean_absolute_error: 0.0929 - val_loss: 1.5865e-04 - val_mean_absolute_error: 0.0126
Epoch 3/50
1408/1425 [============================>.] - ETA: 0s - loss: 0.0196 - mean_absolute_error: 0.0824
Epoch 00003: val_loss improved from 0.00001 to 0.00001, saving model to ./model_a_checkpoint/09112020 1115h.h5
1425/1425 [====================

# Model B (word vectors + price history)

In [82]:
model_b_df = df.dropna(subset=['cleaned_text', 'prev_30mins_prices'])

model_b_X = model_b_df.loc[:, ['cleaned_text', 'prev_30mins_prices']]
model_b_y = model_b_df.loc[:, '5mins_price_diff_perc']

model_b_X_train, model_b_X_test, model_b_y_train, model_b_y_test = train_test_split(model_b_X, model_b_y, test_size=0.33, random_state=42)

model_b_X_train_text = model_b_X_train.iloc[:, 0]
model_b_X_train_price_history = model_b_X_train.iloc[:, 1]
model_b_X_test_text = model_b_X_test.iloc[:, 0]
model_b_X_test_price_history = model_b_X_test.iloc[:, 1]
                           
model_b_corpus_list = []

for i in model_b_X_train_text:
    model_b_corpus_list.append(i.split())
    
model_b_word2vec_model = Word2Vec(model_b_corpus_list, min_count=1, size=100)
model_b_pretrained_weights = word2vec_model.wv.vectors

model_b_num_words = [len(i) for i in model_b_corpus_list]
model_b_longest_sentence_len = max(model_b_num_words)

model_b_X_train_padded = sentence_to_indices_padded(model_b_X_train_text, model_b_longest_sentence_len)
model_b_X_test_padded = sentence_to_indices_padded(model_b_X_test_text, model_b_longest_sentence_len)

model_b_X_train_input = [model_b_X_train_padded, model_b_X_train_price_history]
model_b_X_test_input = [model_b_X_test_padded, model_b_X_test_price_history]

In [86]:
model_b_X_train

,cleaned_text,prev_30mins_prices
286,happy birthday theleegreenwoodflashbackfriday,"[257.65869642857143, 257.658619047619, 257.658..."
824,tomorrow the house votes on kateslaw amp no sa...,"[243.66, 243.67, 243.7, 243.65, 243.63, 243.65..."
75,stocks and the economy have a long way to go a...,"[267.475, 267.47, 267.49, 267.49, 267.52, 267...."
2133,a year ago the pundits amp talking heads peopl...,"[279.12, 279.16, 279.09, 279.08, 279.05, 279.0..."
1947,looking more amp more like the trump campaign ...,"[279.3987916666666, 279.39869047619044, 279.39..."
...,...,...
1720,jon kyl will be an extraordinary senator repre...,"[264.12, 264.15, 264.23, 264.33, 264.215000000..."
1151,i am so proud of my daughter ivanka to be abus...,"[257.81647058823535, 257.82058823529405, 257.8..."
1186,nancy pelosi and fake tears chuck schumer held...,"[227.6575, 227.66125, 227.665, 227.66875, 227...."
1364,all will stay in mexico if for any reason it b...,"[265.26937853107347, 265.27025423728816, 265.2..."


In [93]:
def create_model_b(pretrained_weights, longest_sentence_len, price_history_shape):
    vocab_size, embedding_size = pretrained_weights.shape
    
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False)(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='relu')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [94]:
model_b = create_model_b(model_b_pretrained_weights, model_b_longest_sentence_len, (30,1,))

In [95]:
model_b.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [96]:
model_b.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_index_input (InputLaye [(None, 157)]        0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 157, 100)     589300      sentence_index_input[0][0]       
__________________________________________________________________________________________________
price_history_input (InputLayer [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
model1_LSTM1 (LSTM)             (None, 157, 4)       1680        embedding_14[0][0]               
____________________________________________________________________________________________

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_b_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a.fit(model_b_X_train_input, model_b_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

Train on 1352 samples, validate on 339 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
1344/1352 [============================>.] - ETA: 0s - loss: 0.0296 - mean_absolute_error: 0.1076
Epoch 00001: val_loss improved from inf to 0.00004, saving model to ./model_b_checkpoint/09112020 1200h.h5
1352/1352 [==============================] - 21s 16ms/sample - loss: 0.0297 - mean_absolute_error: 0.1078 - val_loss: 4.4453e-05 - val_mean_absolute_error: 0.0067
Epoch 2/50
1344/1352 [============================>.] - ETA: 0s - loss: 0.0216 - mean_absolute_error: 0.0891
Epoch 00002: val_loss improved from 0.00004 to 0.00000, saving model to ./model_b_checkpoint/09112020 1200h.h5
1352/1352 [==============================] - 16s 12ms/sample - loss: 0.0216 - mean_absolute_error: 0.0890 - val_loss: 6.6148e-07 - val_mean_absolute_error: 7.2531e-04
Epoch 3/50
1344/1352 [============================>.] - ETA: 0s - loss: 0.0167 - mean_absolut